In [1]:
%load_ext autoreload
%autoreload 2

In [181]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import grammarlearner as gl


## Recurrent Neural Networks

In this notebook, you will train a recurrent neural network in PyTorch to learn an artificial grammars. The goal of this assignment is to build intuition about the training and operation of recurrent neural networks. We will use a Simple Recurrent Network (SRN) as proposed in Elman (1990). Conceptually, SRNs are similar to the very popular LSTM and GRU networks in contemporary machine learning. We certainly could use more these complex types of recurrent networks instead, but we don't need the additional power for our purposes here.

For this notebook, you'll be using the code in `grammarlearner.py`, as well as adding your own code to that module.



The original research that this assignment is based on was reported in Servan-Schreiber, Cleeremans, and McClelland (1991) as an investigation into the properties of recurrent neural networks. The SRN has also been studied as a model of implicit sequence learning in humans, which was also investigated with the Reber grammar (Cleeremans and McClelland, 19991). Feel free to take a look at these references to learn more.

** References: **
* Cleeremans, A. and McClelland, J. L. (1991). Learning the structure of event sequences. J Exp Psychol Gen, 120:235–253. 
* Elman, J. L. (1990). Finding structure in time. Cognitive Science, 14:179–211. 
* Servan-Schreiber, D., Cleeremans, A., and McClelland, J. L. (1991). Graded state machines: The representation of temporal contingencies in simple recurrent networks. Machine Learning, 7:161–193.

<div class="alert alert-info">
Some of this assignment code was adapted from Brendan Lake's course on computational cognitive modeling. Thanks for sharing!
</div>

## Reber grammar

In this assignment, your network will be learning the "Reber grammar" (Reber, 1976), a classic learning task in pyschology. 

<img src="images/reber.png" style="width: 500px;"/>

The Reber grammar defines a set of allowable or "grammatical" sequences. The SRN will learn a sequence prediction task: what is the next element/symbol in a sequence, given the past elements. If the SRN predictions are aligned with the grammar, we can say that the SRN has implicitly learned to behave like the grammar, although without explicitly learning rules!

The Reber grammar is diagrammed above as a finite state machine (FSM) with six nodes (states) indicated by \#0,\#1,...,\#5. The FSM creates a string by traversing a path through the graph and visiting various nodes. It can transition between nodes that are connected by a directed edge, and when traversing an edge the FSM emits the output symbol associated with that edge. In the Reber grammar, each string begins with 'B' and ends with 'E'.

Let's go over an example. Starting at 'Begin', let's trace a path through the following nodes, \#0, \#1, \#3, and \#5, now ending at 'End.' This would create the string 'BTXSE' (please confirm for yourself). Other paths are possible, and self-connections and loops allow the FSM to create an infinite set of strings. For example, we can retrace the same path as before, but following the self-connection at node \#1, to create the path \#0, \#1, \#1, \#3, and \#5. This creates the string 'BTSXSE.' Likewise, we could keep adding loops, creating 'BTSSXSE', 'BTSSSXSE', ...., 'BTSSSSSSSXSE' and so on. All of these strings are considered "grammatical" according to the Reber grammar. A string that cannot be produced by the grammar is called "ungrammatical."

Here are some initial exercises to check your understanding of the Reber grammar.

<div class="alert alert-success" role="alert">
<h3> Problem 1 </h3>
Which of the following strings are grammatical? For each string below, please write 'YES' (for grammatical) or 'NO' (for ungrammatical)
<ul>
<li>BTSSXSSE</li>
<li>BTXXVPSE</li>
<li>BTXXVPXVPSE</li>
<li>BTXXXVPXVPSE</li>
<li>BTXXTVPXVPSE</li>
</ul>
<br>
Write your answers in the Markdown cell below this one.
</div>

- BTSSXSSE
  - No (no double S at the end)
- BTXXVPSE
  - Yes
- BTXXVPXVPSE
  - Yes
- BTXXXVPXVPSE
  - No tripple X
- BTXXTVPXVPSE
  - Yes

<div class="alert alert-success" role="alert">
<h3>Problem 2 </h3>

The Reber grammar was carefully designed to display some interesting qualities to make it a difficult learning task. In general, the best a learner can do is predict one of two possible successors for a given node (except at the end of the sequence). For instance, if the grammar is at state \#1, both 'S' and 'X' are valid next symbols.
<br><br>
Note that, except for the special beginning and end symbols 'B' and 'E', *each symbol appears in two different places (on different edges) in the grammar.* Therefore, for a learner aiming to master the grammar and make the best  possible predictions, she cannot *just* pay attention to the previous symbol. This strategy does not uniquely identify the right set of possible next symbols. Instead, she must track the history further back, in order to make optimal predictions.
<br><br>
In this problem, your job is to simulate a "first-order" memory predictor, meaning you can only remember one symbol back (we also call this the "first-order statistics" of the domain). For each of these symbols, {'B', 'T', 'S', 'X', 'P', 'V'}, what is the set of possible successors given just a first-order memory?
<br><br>
Write your answers in the Markdown cell below this one.
</div>


|Last Symbol | Possuble Future Symbols|
|:------------:|:-------------------------|
|B | T, P
|T | T, S, X, V
|S | S, X, E
|X | X, S, T, V
|P | T, V, S
|V | P, V, E

## Loading the data

In our simulations, we are going to limit consideration to all the strings that are grammatical, with a limit on length of 10 symbols or less. There are 43 such strings, divided into a 21 strings [training set](data/reber_train.txt) and a 22 strings [test set](data/reber_test.txt) (Servan-Schreiber et al., 1991). Follow the links to see the training and test sets.

To train the network, we'll need to load some training and test data, and convert it into an appropropriate format for the network. In `grammarlearner.py`, there's a function for that conversion: `seq_to_tensor` takes a sequence of symbols and converts them to a 3D PyTorch tensor of size (seq_length x 1 x n_letters). The first dimension iterates over each symbol in the sequence, and each symbol is encoded as a one-hot vector (a 1 x n_letters tensor) indicating which symbol is present.

Read through `grammarlearner.py`, and then in the cell below, create a list named `training_input` that has as items the tensors for each string in the training set (which is located at `data/reber_train.txt`). Use the functions already defined in `grammarlearner.py` to make your task easier. Then print out the string for the final training pattern as well as its tensor representation. Explain to yourself why the tensor looks the way it does.

In [73]:
contents = gl.load_data('data/reber_train.txt')
all_letters = 'BTSXPVE'
training_input = [gl.seq_to_tensor(c, all_letters) for c in contents]
for i in range(3):
    print(contents[i], training_input[i])

BTSXSE tensor([[[1., 0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 1.]]])
BTXSE tensor([[[1., 0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 1.]]])
BTSSSXSE tensor([[[1., 0., 0., 0., 0., 0., 0.]],

        [[0., 1., 0., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 1.]]])



## SRN architecture 

<img src="images/srn.png" style="width: 400px;"/>

This is the SRN architecture. The goal of the SRN is to predict the next symbol in the sequence, based on previous sequence of symbols it is provided. For instance, it may read 'B', then it may read 'T', and then try to predict the next element which could be either 'S' or 'X', as defined by the Reber grammar. 

The SRN has an input, hidden, and output layer. Each unit in the input and output layer corresponds to a different symbols (see Figure). We use a softmax output layer to ensure that the network's prediction is a valid probability distribution over the set of possible symbols which could occur next.

The `SRN` class processes a sequence (and makes predictions) one symbol at a time. Since the hidden layer is recurrent, the hidden activations from the previous symbol are passed to the `forward` method as the tensor `hidden`, along with the next symbol to be processed as the tensor `input` (The previous hidden state is shown as the "Context units" in the above diagram). The output is the predicted probability of the next symbol (in the code below, `output` is the log-probability vector).
<div class="alert alert-info">
Note: The ordering of the symbols in the picture above may not match the order you're using to encode your one-hot vectors; thats's fine as long as you're consistent with the ordering you use.

I'm presenting the PyTorch code in the notebook to help you walk through it - we could alternatively have put it in `grammarlearner.py`.
</div>

In [207]:
class SRN(nn.Module):
    
    def __init__(self, nsymbols, hidden_size):
        '''
        Initialize the network. 
        nsymbols is the number of possible different letters
        and hidden_size is the number of hidden units.
        '''
        super(SRN, self).__init__()
        self.hidden_size = hidden_size
        # Try to explain to yourself the two lines below -
        # why do they have the sizes that they do?
        self.i2h = nn.Linear(nsymbols + hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, nsymbols)
        self.softmax = nn.LogSoftmax(dim=1)
        self.epoch = 0 #keep track of the training

    def forward(self, input, hidden):
        '''
        Process the given input, assuming that the hidden nodes
        have the activations given in hidden.
        
        input: 1 x nsymbols tensor, with one-hot encoding of a single symbol
        hidden: 1 x hidden_size tensor, which is the previous hidden state
        
        Note: If you have an SRN object named rnn and you call
        rnn(input, hidden), you are implicitly calling the forward function.
        We never call forward directly but instead call the object
        (See note here: https://pytorch.org/docs/stable/nn.html#torch.nn.Module.forward)
        '''
        combined = torch.cat((input, hidden), 1) # tensor size 1 x (nsymbol + hidden_size)
        hidden = self.i2h(combined) # 1 x hidden size
        hidden = torch.sigmoid(hidden)
        output = self.h2o(hidden) # 1 x nsymbol
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        '''
        Get zero activations that is the size of the hidden nodes. For the first
        input in a sequence, the hidden activation is all zeros.
        '''
        return torch.zeros(1, self.hidden_size)

## Function for training the network

The SRN class defines a forward function that we'll use as part of training the network, but we'll need to write a function to actually go through the training data and to call the appropriate functions to compute the gradients and adjust the weights.

The cells in this section will walk you through implementing this function (`train_srn`) and a related helper function (`train_pattern`), as well as calling the train function.

<div class="alert alert-success">The first function you'll write is a helper function, `train_pattern`. This function is described in a comment in `grammarlearner.py`, and parts of the function are implemented for you. Finish implementing the function. You'll test your implementation later in this section.
</div>

<div class="alert alert-success">Now, you'll write `train_srn`. Again, this function is described in a comment in `grammarlearner.py`. Implement the function. You'll test your implementation later in this section.
</div>

<div class="alert alert-success">Okay, now you've written the functions to train the SRN, but we need to know what to call it with. The code below initailizes an optimizer and loss function; you can take a look at the [PyTorch documentation](https://pytorch.org/docs/stable/index.html) if you want to learn more about these. Modify the cell below as directed in the comments, and then run it - you'll train a network!

After the 400 epochs, training loss should be around 0.60 (there are random elements in the training, so it may not be 0.60 exactly, but it should be close to that). The loss should be printed every 20 epochs by your `train_srn` function. 

If you run into errors, look back at your implementations in the past two parts, adding print statements to help you debug and asking for help in office hours or on Piazza if you get stuck.

In [296]:
nletters = 7 # How many total letters there are - change this line!
nhidden = 8  # number of hidden units - you'll experiment with changing this
nepochs = 300 # number of passes through the entire training set 
learning_rate = 0.01

rnn = SRN(nletters,nhidden) # Change this line to call the SRN constructor and initialize the network
# Uncomment the line below once you've fixed the line above (it crashes if rnn = None)
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate) # stochastic gradient descent
criterion = nn.NLLLoss() #log-likelihood loss function


train = gl.load_data('data/reber_train.txt')
test = gl.load_data('data/reber_test.txt')

all_strings = [] # Change this so strings_all represents a list of all of the strings in training and test
all_letters = 'BTSXPVE' # Change this so all_letters represents a string of all letters in the grammar, in the
                 # order you're performing you're using elsewhere for one-hot encodings

# Add code below to load the data and call the train_srn function that you wrote
# in grammarlearner


gl.train_srn(train, test, nepochs, rnn, optimizer, criterion, eval_single_output_fn = eval_output_three_param)

Epoch: 0
	Total Loss:	 47.2021468041
	Training Accuracy:	34.21%
	Testing Accuracy:	40.52%
Epoch: 20
	Total Loss:	 44.6707892274
	Training Accuracy:	34.21%
	Testing Accuracy:	40.52%
Epoch: 40
	Total Loss:	 39.1044488706
	Training Accuracy:	59.15%
	Testing Accuracy:	63.82%
Epoch: 60
	Total Loss:	 33.4415589677
	Training Accuracy:	73.55%
	Testing Accuracy:	74.6%
Epoch: 80
	Total Loss:	 28.0692638484
	Training Accuracy:	77.42%
	Testing Accuracy:	78.19%
Epoch: 100
	Total Loss:	 23.8203176657
	Training Accuracy:	77.42%
	Testing Accuracy:	78.19%
Epoch: 120
	Total Loss:	 21.2825425188
	Training Accuracy:	89.89%
	Testing Accuracy:	89.84%
Epoch: 140
	Total Loss:	 19.3792659988
	Training Accuracy:	89.89%
	Testing Accuracy:	89.84%
Epoch: 160
	Total Loss:	 17.5781096175
	Training Accuracy:	93.04%
	Testing Accuracy:	94.28%
Epoch: 180
	Total Loss:	 16.0710498528
	Training Accuracy:	100.0%
	Testing Accuracy:	100.0%
Epoch: 200
	Total Loss:	 14.9617666498
	Training Accuracy:	100.0%
	Testing Accuracy:	10

### Evaluating accuracy

The network's loss provides one measure of how well it's performing, but it isn't all that intuitive to interpret. The loss function being used is the log-likelihood ([you can read about it in the PyTorch documentation](https://pytorch.org/docs/stable/nn.html#torch.nn.NLLLoss)), so the network incurs loss even if its highest probability output is correct, as long as the other outputs also have some probability. It doesn't take into account that fact that sometimes, two different outputs could be correct according to the grammar: it's only looking at the current sequence for correctness information.

You, as a person, might want to know how often the network predicts a letter that is accurate according to the grammar. In this section, you'll  add to some code that I've started to calculate that, and modify your `train_srn` function to also print out train and test accuracy.


<div class="alert alert-success">
The only function you'll implement is a helper function, `eval_pattern`. This function is described in a comment in `grammarlearner.py`, and parts of the function are implemented for you. Finish implementing the function. You'll test your implementation later in this section.
</div>

The `eval_pattern` function is called from the function `eval_set` (already implemented). Here's the function header for eval_set:
`eval_set(list_seq, rnn, all_letters, eval_single_output_fn)`

`list_seq` is a list of sequences that we want to predict, and `eval_set` just calls `eval_pattern` on each sequence in the list and returns the average percent correct. This will allow us to calculate the average accuracy for the entire training (or test) set easily!

But what should we be putting in for `eval_single_output_fn`? According to the comments, this is the same kind of function as in the parameters of `eval_pattern`: a function that takes three parameters - the output of the network for predicting the next symbol, the letters so far in the sequence, and all_letters - and returns True if the prediction is correct and False if the prediction is incorrect.

In `grammarlearner.py`, an implementation of this type of function for the Reber grammar is provided for you: `eval_single_output_reber`. However, this function takes four parameters, not three. The fourth parameter is just the list of all strings possible in the grammar. To get around this, we can make a version of the function that already has that argument provided, since it's going to be the same for all the times we need to invoke the function:

In [280]:
strings_all = gl.load_data('data/all_reber_strings.txt')
eval_output_three_param = lambda output, preceding_letters, all_letters, verbose=False: \
                                grammarlearner.eval_single_output_reber(output, 
                                                                        preceding_letters, 
                                                                        all_letters,
                                                                        strings_all,
                                                                       verbose)

The second line in the cell above is a [lambda expression](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) and it creates a new function, called `eval_output_three_param`. The new function only takes three arguments, labeled here as `output`, `preceding_letters`, and `all_letters`, and it returns the result of running `grammarlearner.eval_single_output_reber` with the arguments `output`, `preceding_letters`, `all_letters`, and `strings_all`, which is defined in the first line in the cell above. Now, `eval_output_three_param` is a function, which you can call like any other function, and it has exactly the form we need to pass to `eval_set`. 

(Note: As you've seen in past problem sets, to pass a function as an argument, you don't include parentheses, so your call to `eval_set` will look something like `eval_set(list_seq, rnn, all_letters, eval_output_three_param)`.

<div class="alert alert-success">
Modify `train_srn` so that it prints out the training and testing accuracy every time it prints out the training loss. Assume that the argument eval_single_output_fn is of the form expected by `eval_set`.
<br>
There are an infinite number of possible strings in the Reber grammar, so we can't actually pass in "all possible strings in the grammar." However, we know that we'll be making predictions based on histories of length no more than 9. Thus, if we include all possible grammatrical strings *or* prefixes of grammatical strings  of length 10 or fewer, then we've covered the possibly correct predictions. A file with all of these strings is located in `'data/all_reber_strings.txt'`.
<br>
Once you've modified `train_srn`, modify your cell above that trains the network to pass in a version of `eval_single_output_reber` that only takes three inputs. Then, re-run your cell above to train the network. The accuracy on the training and test set should be pretty good (above 95% correct for each).
<br>
Again, if you run into errors, look back at your implementation of `eval_pattern`, adding print statements to help you debug and asking for help in office hours or on Piazza if you get stuck.
<br>
(Note: We alternatively could have passed in the Reber grammar itself in some form, rather than the possible strings, to evaluate accuracy.)
</div>

## Visualizing how a sequence is processed
Now let's understand what the network has learned!

To do that, we'll want to print out the activations of the hidden layer as we process a sequence and print the output probabilities. For example, if we were evaluating a test string that started with `B`, we might want to start by displaying something like: 

```
Current input symbol: B
  Input : B:1.00 E:0.00 P:0.00 S:0.00 T:0.00 V:0.00 X:0.00 
Predicted next symbol:
  Hidden: 0.21 0.13 0.06 0.65 0.93 0.60 0.15 0.95 
  Output: B:0.00 E:0.00 P:0.42 S:0.00 T:0.57 V:0.01 X:0.00 
```
  
The 'Input' pattern shows the one-hot encoding of 'B'.    
The 'Hidden' pattern shows the activation of the hidden layer.   
The 'Output' pattern shows the prediction, in terms of probabilities of each of the successor symbols.  

In this case, the network is splitting its guess between 'P' and 'T' for predicting the next symbol. Does this make sense given the Reber grammar? Yes! From 'B', both 'P' and 'T' are valid symbols, and no other symbols are valid. Note that the network will not always choose exactly 50/50 when predicting successors, since the training patterns are not perfectly balanced in this way. (You might think about whether people would be similarly biased based on what they saw when learning the grammar.

(Note: The ordering of my one-hot encoding, the hidden activations, and output probabilities may all differ somewhat from what you got - training is stochastic, so different results occur each time!)




<div class="alert alert-success" role="alert">

Finish implementing the function `eval_viz` in `grammarlearner.py` in order to visualize the processing of a sequence.
<br>
Then call `eval_viz` in the cell below to visualize each step of evaluating the test sequence `'BPVPSE'`. Does the network make the right predictions at each step? In a markdown cell below the code cell in which you call `eval_viz`, write two or three sentences about the performance and your reaction as to what's been learned or not learned.

</div>

In [297]:
grammarlearner.eval_viz('BPVPSE',rnn, all_letters)

Current Input Symbol: 
  Input:	B:1.0 T:0.0 S:0.0 X:0.0 P:0.0 V:0.0 E:0.0 
Predicted Next Symbol: B
  Hidden:	0.54 0.92 0.78 0.41 0.16 0.84 0.33 0.31 
  Output:	B:-0.02 T:-6.2 S:-5.2 X:-7.06 P:-6.43 V:-6.25 E:-5.69 

Current Input Symbol: B
  Input:	B:0.0 T:0.0 S:0.0 X:0.0 P:1.0 V:0.0 E:0.0 
Predicted Next Symbol: T
  Hidden:	0.37 0.82 0.07 0.54 0.92 0.05 0.83 0.05 
  Output:	B:-5.35 T:-0.59 S:-5.61 X:-5.39 P:-0.86 V:-4.73 E:-7.0 

Current Input Symbol: P
  Input:	B:0.0 T:0.0 S:0.0 X:0.0 P:0.0 V:1.0 E:0.0 
Predicted Next Symbol: T
  Hidden:	0.17 0.36 0.24 0.81 0.84 0.77 0.38 0.44 
  Output:	B:-5.1 T:-0.62 S:-3.64 X:-3.99 P:-4.08 V:-0.93 E:-7.24 

Current Input Symbol: V
  Input:	B:0.0 T:0.0 S:0.0 X:0.0 P:1.0 V:0.0 E:0.0 
Predicted Next Symbol: P
  Hidden:	0.94 0.1 0.49 0.88 0.96 0.05 0.02 0.05 
  Output:	B:-8.61 T:-4.09 S:-9.18 X:-9.86 P:-0.47 V:-1.08 E:-3.91 

Current Input Symbol: P
  Input:	B:0.0 T:0.0 S:1.0 X:0.0 P:0.0 V:0.0 E:0.0 
Predicted Next Symbol: X
  Hidden:	0.05 0.21 0.13 

- My network learned that B always starts the string
- My network learned that after B there is a split between 'T' and 'B'
- My network learned when the word was about to end (correctly predicted 'E')
- My network messed up what comes after the P, (it guessed T but it was V) Both are valid answers but it guessed wrong on this one

The network always makes valid guesses but doesn't always guess what the next letter is going to be correctly because there are multiple ways to make valid words and it can't read minds. 

## Analyzing the time course of learning

Cleeremans and McClelland (1991) ran a psychology experiment that studied how people implicitly learn the structure of the Reber grammar. They found that in the initial stages of learning, people's behavior was largely consistent with the first-order statistics, as computed by the first-order memory that you worked out earlier in this problem set. Thus, if the previous symbol was 'V', their behavior would be largely consistent with four possible successors (rather than just 2, if they had mastered the grammar), regardless of what had happened previously in the sequence.

In the problem below, you will examine the timecourse of learning in detail.

<div class="alert alert-success" role="alert">
Retrain the network for only **5 epochs** (set `nepochs = 5` a few cells up). Using the `eval_viz` function and examining different sequences from the **test set**, how would you describe what the network has learned? Please write your answer in one or two paragraphs, using specific examples. Mention first-order statistics (or higher-order statistics) if appropriate.
<br><br>
*Hint: It is helpful to examine a sequence that has the same symbol appearing twice, but where the Reber grammar is visiting different nodes (e.g., 'P' on the way to visiting node \#2, and also 'P' on the way to visiting node \#3). Does the network make the same prediction in both places? Does looking at the pattern of hidden activations help? You may also want to write additional code to help you understand in general what kind of predictions the network is making, such as by examining what possibilities the network predicts after each letter (i.e., you'd be printing out the first order statistics). You could make a versions of `eval_set`/`eval_pattern` that outputs this type of information. *
<br><br>
Write your answers in the cell below this one.
</div>

This model knows basically nothing. It overpredicts S and T which is probably a function of those letters can loop in the grammar so they probably appear most often. It's hard to know what it actually knows vs what it is guessing because it isn't trained enough. One of the bigges things to notice is the the weights of the hidden layers are so much weaker compared to the weights of the input. This means the network is essentially 'ignoring' the information it has already seen.

In [298]:
rnn_dumb = SRN(nletters,nhidden) # Change this line to call the SRN constructor and initialize the network
gl.train_srn(train, test, 5, rnn_dumb, optimizer, criterion, eval_single_output_fn = eval_output_three_param, verbose = True)
print()
grammarlearner.eval_viz(test[0],rnn, all_letters)
print('#############')
grammarlearner.eval_viz(test[1],rnn, all_letters)

Epoch: 0
	Total Loss:	 47.5934527488
	Training Accuracy:	28.38%
	Testing Accuracy:	24.53%
Epoch: 4
	Total Loss:	 47.5934527488
BPTVPXVVE
	    S False
	 B   S False
	 BP   S False
	 BPT   S False
	 BPTV   S False
	 BPTVP   S True
	 BPTVPX   S False
	 BPTVPXV   S False
	 BPTVPXVV   S False
	Training Accuracy:	28.38%
	Testing Accuracy:	24.53%

Current Input Symbol: 
  Input:	B:1.0 T:0.0 S:0.0 X:0.0 P:0.0 V:0.0 E:0.0 
Predicted Next Symbol: B
  Hidden:	0.62 0.95 0.74 0.37 0.16 0.8 0.44 0.26 
  Output:	B:-0.02 T:-6.03 S:-6.79 X:-5.63 P:-4.89 V:-7.96 E:-5.6 

Current Input Symbol: B
  Input:	B:0.0 T:0.0 S:0.0 X:0.0 P:1.0 V:0.0 E:0.0 
Predicted Next Symbol: P
  Hidden:	0.84 0.93 0.09 0.38 0.91 0.03 0.95 0.02 
  Output:	B:-6.25 T:-4.1 S:-8.88 X:-6.13 P:-0.02 V:-8.64 E:-6.8 

Current Input Symbol: P
  Input:	B:0.0 T:1.0 S:0.0 X:0.0 P:0.0 V:0.0 E:0.0 
Predicted Next Symbol: X
  Hidden:	0.25 0.66 0.19 0.36 0.86 0.74 0.95 0.32 
  Output:	B:-4.21 T:-1.48 S:-4.08 X:-0.36 P:-3.16 V:-6.87 E:-6.83 

Cu

<div class="alert alert-success" role="alert">
Retrain the network again. This time train for **50 epochs**. Answer the same questions as above.
<br><br>
Write your answers in the cell below this one.
</div>

In [291]:
rnn_dumb = SRN(nletters,nhidden) # Change this line to call the SRN constructor and initialize the network
gl.train_srn(train, test, 50, rnn_dumb, optimizer, criterion, eval_single_output_fn = eval_output_three_param, verbose = True)
print()
grammarlearner.eval_viz(test[0],rnn, all_letters)
print()
print('########')
print()
grammarlearner.eval_viz(test[1],rnn, all_letters)

Epoch: 0
	Total Loss:	 47.1711202546
	Training Accuracy:	 0.247789115646
	Testing Accuracy:	 0.294047619048
Epoch: 20
	Total Loss:	 47.1711202546
	Training Accuracy:	 0.247789115646
	Testing Accuracy:	 0.294047619048
Epoch: 40
	Total Loss:	 47.1711202546
	Training Accuracy:	 0.247789115646
	Testing Accuracy:	 0.294047619048
Epoch: 49
	Total Loss:	 47.1711202546
BPTTVVE
	    P False
	 B   V False
	 BP   P False
	 BPT   P False
	 BPTT   V True
	 BPTTV   V True
	 BPTTVV   V False
	Training Accuracy:	 0.247789115646
	Testing Accuracy:	 0.294047619048

Current Input Symbol: 
  Input:	B:1.0 T:0.0 S:0.0 X:0.0 P:0.0 V:0.0 E:0.0 
Predicted Next Symbol: S
  Hidden:	1.0 0.0 1.0 0.0 1.0 0.89 0.01 0.0 
  Output:	B:-99.95 T:-46.15 S:0.0 X:-188.31 P:-170.57 V:-118.73 E:-98.85 

Current Input Symbol: B
  Input:	B:0.0 T:0.0 S:0.0 X:0.0 P:1.0 V:0.0 E:0.0 
Predicted Next Symbol: S
  Hidden:	1.0 0.0 1.0 0.0 1.0 0.94 0.0 0.0 
  Output:	B:-100.53 T:-45.8 S:0.0 X:-189.3 P:-171.75 V:-119.19 E:-99.14 

Current

The hidden layer is now a bit bigger which suggests that the network has learned to value the past but it is still orders of magnitude smaller than the output. It has learned that Bs almost never happen (they only happen at the beginning of the string).

<div class="alert alert-success" role="alert">
Retrain the network again. This time train for **100 epochs**. Answer the same questions as above.
</div>

<div class="alert alert-success" role="alert">
Retrain the network again, this time train for the full **400 epochs**. Answer the same questions as above.
<br><br>
You may want to go through this whole process twice, in case there is some variability in your network runs.
</div>

## Optional: Analyzing the dependence on the number of hidden units

<div class="alert alert-success" role="alert">
**(Optional question for extra practice - your answer won't be graded, and you will not lose credit for omitting this question)**
<br>
Above, we kept the number of hidden units fixed and adjusted the amount of training. What would happen if we reduced the number of hidden units? Retrain the network with 300 epochs and only 2 hidden units. How well does the network perform? It still probably gets high training and test accuracy, but it has learned the grammar far less well. How can you tell? Explore the network's behavior to find evidence for what parts of the grammar it hasn't learned or has learned less well. Add cells below to collect your evidence and explain your results.
</div>

In [ ]:
nletters = 0 # How many total letters there are - change this line!
nhidden = 2  # number of hidden units - you'll experiment with changing this
nepochs = 300 # number of passes through the entire training set 
learning_rate = 0.01

rnn = None; # Change this line to call the SRN constructor and initialize the network
# Uncomment the line below once you've fixed the line above (it crashes if rnn = None)
#optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate) # stochastic gradient descent
criterion = nn.NLLLoss() #log-likelihood loss function

all_strings = [] # Change this so strings_all represents a list of all of the strings in training and test
all_letters = "" # Change this so all_letters represents a string of all letters in the grammar, in the
                 # order you're performing you're using elsewhere for one-hot encodings

# Add code below to load the data and call the train_srn function that you wrote
# in grammarlearner
